# Uczenie maszynowe w bezpieczeństwie

## Projekt 1

### Autorzy: Kamil Różalski, Daniel Jończyk, Dominik Borowiec

### Grupa 2ID24A

### Zadanie 1

> Pobrać, rozpakować i przeanalizować strukturę plików i katalogów archiwum zawierającego wiadomości poczty elektronicznej. Dane te dostępne są pod adresem:
> https://plg.uwaterloo.ca/~gvcormac/treccorpus07/
> Uwaga. Nie należy otwierać plików z archiwum ani w przeglądarce HTML ani w programie pocztowym!

W pierwszym zadaniu zajęliśmy się procesem ściągania, dekompresji i analizy archiwum e-mailowego, dostępnego pod adresem https://plg.uwaterloo.ca/~gvcormac/treccorpus07/.

Początkowo, po ściągnięciu archiwum, użyliśmy odpowiednich narzędzi do jego rozpakowania, co pozwoliło nam na dokładne przeanalizowanie struktury zawartości folderów w celu zrozumienia ich organizacji. Wewnątrz archiwum znaleźliśmy liczne pliki, które zawierały treści wiadomości e-mailowych.

Struktura archiwum była podzielona na główne katalogi takie jak data, delay, full i partial, które zawierały różne segmenty wiadomości.

Aby dokładniej zbadać dane, w kolejnych etapach moglibyśmy zastosować metody i techniki przetwarzania języka naturalnego (NLP) do wydobywania kluczowych informacji, klasyfikowania treści oraz analizowania tendencji w komunikacji mailowej.

### Zadanie 2

> Wykorzystując informacje z wykładu oraz stosując technikę zakazanych słów kluczowych (blacklist), dokonać klasyfikacji binarnej wiadomości z archiwum z podziałem na: spam (wiadomości typu spam) oraz ham (wiadomości pożądane).
Uwagi:
> 1. Przed przystąpieniem do procesu klasyfikacji usunąć z wiadomości stopping words (np. the, is, are, . . . ),
dokonać stemizacji słów w wiadomościach oraz ekstrakcji tokenów.
> 2. Do realizacji zadania użyć języka Python oraz bibliotek: string, email, NLTK, os.
> 3. Zbiór zakazanych słów kluczowych powinien być wygenerowany na podstawie danych z podzbioru treningowego,
natomiast ewaluacja danych uzyskanych z podzbioru testowego.
> 4. Wynikiem ewaluacji powinna być macierz konfuzji (procentowa) oraz wartość wskaźnika accuracy, również w
postaci procentowej.

W ramach drugiego zadania skupiliśmy się na klasyfikacji binarnej wiadomości e-mail na podstawie technik prezentowanych w wykładzie, szczególnie koncentrując się na technice zakazanych słów kluczowych (blacklist). Celem było oddzielenie wiadomości typu spam od tych pożądanych (ham).

Na początku procesu klasyfikacji przeprowadziliśmy kilka kluczowych operacji przetwarzania tekstu na danych wiadomościach. Usunęliśmy z nich stopping words, co pozwoliło zredukować zakłócenia w analizie i skupienie na istotnych elementach treści. Następnie dokonaliśmy stemizacji słów, co jest procesem redukcji słów do ich korzeni, oraz ekstrahowaliśmy tokeny, co umożliwiło efektywniejszą analizę i przetwarzanie tekstu.

Do implementacji tego zadania użyliśmy języka Python wraz z bibliotekami takimi jak string, email, NLTK, i os, które są niezbędne do przetwarzania i analizy tekstu. Blacklist, czyli lista zakazanych słów kluczowych, została utworzona na podstawie analizy podzbioru treningowego, co pozwoliło na adekwatne dostosowanie modelu do rzeczywistego charakteru danych.

#### Implementacja

```python
import os
import re
from collections import Counter
from email import message_from_bytes
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score

# Inicjalizacja stemizera i pobranie listy stopping words
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
```

#### Wczytywanie i przetwarzanie e-maili

Funkcja load_and_preprocess_email odczytuje pliki e-mail w formacie binarnym, dekoduje ich zawartość, usuwa znaki interpunkcyjne, konwertuje tekst na małe litery, tokenizuje, usuwa stopping words oraz stemizuje.


#### Wczytywanie etykiet

Funkcja load_labels wczytuje etykiety z pliku, mapując ścieżki wiadomości na etykiety ham (0) lub spam (1).

```python
# Funkcja do wczytywania e-maili i ich przetwarzania
def load_and_preprocess_email(file_path):
    with open(file_path, 'rb') as file:  # zmienione na 'rb' do odczytu binarnego
        email_content = message_from_bytes(file.read())
        # Sprawdzenie, czy treść e-maila jest wieloczęściowa
        if email_content.is_multipart():
            email_body = ' '.join(part.get_payload(decode=True).decode('utf-8', errors='ignore')
                                  for part in email_content.walk()
                                  if part.get_content_type() == 'text/plain')
        else:
            email_body = email_content.get_payload(decode=True).decode('utf-8', errors='ignore')
        # Usunięcie znaków interpunkcyjnych i konwersja na małe litery
        email_body = re.sub(r'[^\w\s]', '', email_body).lower()
        # Tokenizacja
        tokens = word_tokenize(email_body)
        # Usunięcie stopping words i stemizacja
        filtered_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
        return ' '.join(filtered_tokens)


# Funkcja do wczytywania etykiet
def load_labels(label_file_path):
    labels = {}
    with open(label_file_path, 'r') as file:
        for line in file:
            label, message_path = line.strip().split(' ')
            message_id = os.path.basename(message_path)
            labels[message_id] = 0 if label == 'ham' else 1
    return labels
```

Kod wczytuje etykiety i przetwarza e-maile, następnie dzieli dane na zbiory treningowy i testowy.

```python
# Ścieżki do folderów i plików
data_dir = 'trec07p/data'
index_file = 'trec07p/full/index'

# Wczytanie etykiet
labels = load_labels(index_file)

# Wczytanie i przetwarzanie e-maili
emails = []
y = []
for file_name in os.listdir(data_dir):
    file_path = os.path.join(data_dir, file_name)
    if file_name in labels:
        emails.append(load_and_preprocess_email(file_path))
        y.append(labels[file_name])

# Podział na zbiory treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(emails, y, test_size=0.2, random_state=42)
```

Użycie CountVectorizer do transformacji tekstu na wektory cech, a następnie klasyfikacja za pomocą MultinomialNB.

Model jest ewaluowany za pomocą macierzy konfuzji i wskaźnika accuracy, które dostarczają informacji o skuteczności modelu.

```python
# Tworzenie wektora cech za pomocą CountVectorizer
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)

# Klasyfikacja za pomocą MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_counts, y_train)
X_test_counts = vectorizer.transform(X_test)
y_pred = clf.predict(X_test_counts)

# Ewaluacja modelu
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
print(f'Macierz konfuzji:\n{cm}')
print(f'Wartość wskaźnika accuracy: {acc * 100:.2f}%')
```

#### Podsumowanie

Kod z powodzeniem przetwarza i klasyfikuje e-maile na spam i ham wykorzystując technikę zakazanych słów kluczowych oraz zestaw operacji przetwarzania tekstu. Po dokładnym przetworzeniu danych, treningu i testowaniu modelu klasyfikacyjnego, uzyskano następujące wyniki:

    Macierz konfuzji:
        Prawdziwie pozytywne (True Positives, TP) dla wiadomości ham: 4851
        Fałszywie negatywne (False Negatives, FN) dla wiadomości ham: 151
        Prawdziwie pozytywne (TP) dla wiadomości spam: 9820
        Fałszywie pozytywne (False Positives, FP) dla wiadomości spam: 262

    Wartość wskaźnika accuracy: 97.26%

Wysoki wynik accuracy wskazuje na skuteczność modelu w rozróżnianiu między spamem a pożądanymi wiadomościami. Macierz konfuzji również demonstruje, że model ma dobrą zdolność do poprawnego identyfikowania zarówno spamu, jak i hamu, choć zauważalna jest niewielka tendencja do błędnego klasyfikowania nielicznych wiadomości ham jako spam.

### Zadanie 3

> Zweryfikować wpływ stemizacji na pracę algorytmu zadania drugiego a następnie porównać uzyskane wyniki.

Do wykonania zadania zmodyfikowano istniejący kod z zadania drugiego, dodając możliwość opcjonalnego użycia stemizacji w procesie przetwarzania e-maili. Następnie porównano wyniki uzyskane z użyciem stemizacji i bez niej, aby ocenić, jak stemizacja wpływa na dokładność klasyfikacji.

#### Implementacja

Kod składa się z kilku kluczowych części, które umożliwiają wczytywanie, przetwarzanie i klasyfikację danych e-mailowych. Poniżej opisano szczegółowo każdą funkcję oraz jej kluczowe parametry.

```python
import os
import re
from email import message_from_bytes
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score

# Inicjalizacja stemizera i pobranie listy stopping words
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
```

**load_and_preprocess_email(file_path, use_stemming=False)**

    Parametry:
        file_path: ścieżka do pliku e-mail.
        use_stemming: flaga decydująca o użyciu stemizacji (domyślnie wyłączona).
    Opis: Funkcja wczytuje e-mail, dekoduje jego zawartość, usuwa znaki interpunkcyjne, konwertuje tekst na małe litery, tokenizuje go, opcjonalnie stosuje stemizację i filtruje stopping words. Zwraca przetworzony tekst jako ciąg tokenów.

**load_labels(label_file_path)**

    Parametry:
        label_file_path: ścieżka do pliku z etykietami.
    Opis: Funkcja wczytuje etykiety z pliku, przyporządkowując każdej wiadomości etykietę 0 (ham) lub 1 (spam) na podstawie ścieżki do pliku e-mail.

```python
# Funkcja do wczytywania e-maili i ich przetwarzania
def load_and_preprocess_email(file_path, use_stemming=False):
    with open(file_path, 'rb') as file:
        email_content = message_from_bytes(file.read())
        if email_content.is_multipart():
            email_body = ' '.join(part.get_payload(decode=True).decode('utf-8', errors='ignore') 
                                  for part in email_content.walk() 
                                  if part.get_content_type() == 'text/plain')
        else:
            email_body = email_content.get_payload(decode=True).decode('utf-8', errors='ignore')
        email_body = re.sub(r'[^\w\s]', '', email_body).lower()
        tokens = word_tokenize(email_body)
        if use_stemming:
            filtered_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
        else:
            filtered_tokens = [word for word in tokens if word not in stop_words]
        return ' '.join(filtered_tokens)

# Funkcja do wczytywania etykiet
def load_labels(label_file_path):
    labels = {}
    with open(label_file_path, 'r') as file:
        for line in file:
            label, message_path = line.strip().split(' ')
            message_id = os.path.basename(message_path)
            labels[message_id] = 0 if label == 'ham' else 1
    return labels

# Ścieżki do folderów i plików
data_dir = 'trec07p/data'
index_file = 'trec07p/full/index'

# Wczytanie etykiet
labels = load_labels(index_file)

# Wczytanie i przetwarzanie e-maili
emails = []
y = []
for file_name in os.listdir(data_dir):
    file_path = os.path.join(data_dir, file_name)
    if file_name in labels:
        emails.append(load_and_preprocess_email(file_path, use_stemming=False))
        y.append(labels[file_name])

# Podział na zbiory treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(emails, y, test_size=0.2, random_state=42)

# Tworzenie wektora cech za pomocą CountVectorizer
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)

# Klasyfikacja za pomocą MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_counts, y_train)
X_test_counts = vectorizer.transform(X_test)
y_pred = clf.predict(X_test_counts)

# Ewaluacja modelu
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
print(f'Macierz konfuzji:\n{cm}')
print(f'Wartość wskaźnika accuracy: {acc * 100:.2f}%')
```

#### Wyniki

Otrzymane wyniki z tego eksperymentu są następujące:

    Macierz konfuzji:
        Prawdziwie pozytywne dla ham: 4854
        Fałszywie pozytywne dla ham: 148
        Prawdziwie pozytywne dla spam: 9805
        Fałszywie pozytywne dla spam: 277
    Wartość wskaźnika accuracy: 97.18%

#### Podsumowanie

Wprowadzenie opcji stemizacji nie przyniosło znaczącej poprawy dokładności klasyfikacji. Wynik accuracy zmniejszył się nieznacznie z 97.26% do 97.18%. Oznacza to, że dla tego konkretnego zestawu danych i zastosowanego algorytmu, stemizacja nie miała istotnego wpływu na poprawę wyników, a nawet mogła nieznacznie pogorszyć skuteczność klasyfikacji. Wynika to prawdopodobnie z faktu, że redukcja słów do ich korzeni mogła spowodować utratę pewnych niuansów językowych, które są istotne w kontekście rozróżniania spamu od pożądanych wiadomości. Może to sugerować, że dla bardziej złożonych analiz językowych, pełne formy słów mogą dostarczać cenniejszych informacji niż ich zredukowane wersje.

### Zadanie 4

> Dokonać klasyfikacji binarnej wiadomości z archiwum (zadanie 1) na spam i ham, stosując algorytmy rozmytego haszowania.
Uwagi:
> 1. Do tego celu użyć algorytmu LSH (MinHash, MinHashLSH) z biblioteki datasketch.
> 2. Wyniki pracy algorytmu przedstawić przy pomocy procentowej macierzy konfuzji i wskaźnika accuracy.
> 3. Sprawdzić pracę programu dla różnych wartości parametru threshold funkcji MinHashLSH.
> 4. Porównać uzyskane wyniki z wynikami z poprzednich zadań.

Zadanie koncentruje się na klasyfikacji binarnej wiadomości e-mail na spam i ham przy użyciu algorytmów rozmytego haszowania, specyficznie przez implementację algorytmu LSH (Locality-Sensitive Hashing) z wykorzystaniem MinHash z biblioteki datasketch. Celem jest zbadanie, jak różne wartości progu (threshold) wpływają na wyniki klasyfikacji i porównanie tych wyników z rezultatami uzyskanymi w poprzednich zadaniach.

#### Implementacja

```python
import os
import re
from datasketch import MinHash, MinHashLSH
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

# Inicjalizacja listy stopping words
stop_words = set(stopwords.words('english'))

# Funkcja do wczytywania e-maili i ich przetwarzania
def preprocess_email(email_content):
    # Usunięcie znaków interpunkcyjnych i konwersja na małe litery
    email_body = re.sub(r'[^\w\s]', '', email_content).lower()
    # Tokenizacja
    tokens = word_tokenize(email_body)
    # Usunięcie stopping words
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

# Funkcja do wczytywania etykiet
def load_labels(label_file_path):
    labels = {}
    with open(label_file_path, 'r') as file:
        for line in file:
            label, message_path = line.strip().split(' ')
            message_id = os.path.basename(message_path)
            labels[message_id] = 0 if label == 'ham' else 1
    return labels

# Ścieżki do folderów i plików
data_dir = 'trec07p/data'
index_file = 'trec07p/full/index'

# Wczytanie etykiet
labels = load_labels(index_file)

# Wczytanie i przetwarzanie e-maili
emails = []
y = []
for file_name in os.listdir(data_dir):
    file_path = os.path.join(data_dir, file_name)
    if file_name in labels:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
            email_content = file.read()
        emails.append(preprocess_email(email_content))
        y.append(labels[file_name])

# Podział na zbiory treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(emails, y, test_size=0.2, random_state=42)

# Inicjalizacja LSH
lsh = MinHashLSH(threshold=0.5, num_perm=128)
```

W zadaniu 4 każdy przetworzony email jest konwertowany na sygnaturę MinHash przed dodaniem do struktury LSH. Jest to kluczowe dla działania algorytmu rozmytego haszowania, które polega na porównywaniu sygnatur hashujących, aby szybko znaleźć podobne obiekty:

```python
# Tworzenie sygnatur MinHash dla e-maili i dodawanie do LSH
for i, email_tokens in enumerate(X_train):
    m = MinHash(num_perm=128)
    for token in email_tokens:
        m.update(token.encode('utf-8'))
    lsh.insert(str(i), m)
```

#### Zmodyfikowana funkcja klasyfikacji

Funkcja classify_email używa LSH do wyszukiwania podobnych e-maili na podstawie ich sygnatur MinHash. Klasyfikuje e-mail jako ham (0) lub spam (1) na podstawie najczęściej występujących etykiet wśród podobnych e-maili. To stanowi zasadniczą różnicę w podejściu do klasyfikacji w porównaniu do wcześniejszych zadań, gdzie używano bezpośrednio modeli klasyfikacyjnych z sklearn:

```python
# Klasyfikacja e-maili
def classify_email(email_tokens, lsh):
    m = MinHash(num_perm=128)
    for token in email_tokens:
        m.update(token.encode('utf-8'))
    matches = lsh.query(m)
    if matches:
        # Jeśli istnieją podobne e-maile, sprawdź ich etykiety
        labels = [y_train[int(match)] for match in matches]
        return max(set(labels), key=labels.count)
    else:
        return 0  # Domyślnie klasyfikuj jako ham, jeśli nie ma podobnych

# Klasyfikacja zbioru testowego
y_pred = [classify_email(email, lsh) for email in X_test]

# Ewaluacja modelu
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
print(f'Macierz konfuzji:\n{cm}')
print(f'Wartość wskaźnika accuracy: {acc * 100:.2f}%')
```

#### Wyniki

Macierz konfuzji:
[[4992   10]
 [1679 8403]]
Wartość wskaźnika accuracy: 88.80%


#### Podsumowanie

    - Model wykazał bardzo wysoką skuteczność w identyfikacji wiadomości pożądanych (ham), z bardzo niskim poziomem fałszywych pozytywów (FP), co stanowi zaledwie 10 przypadków na 5002. Oznacza to, że tylko nieliczne pożądane wiadomości zostały błędnie zaklasyfikowane jako spam.

    - Widać znaczne wyzwanie w poprawnym klasyfikowaniu wiadomości jako spam, co manifestuje się liczbą 1679 fałszywie negatywnych wyników. Oznacza to, że te wiadomości, które były spamem, nie zostały prawidłowo zidentyfikowane przez model.

    - Wskaźnik accuracy wynoszący 88.80% wskazuje na stosunkowo dobry ogólny poziom klasyfikacji. Jednakże, duża liczba fałszywie negatywnych wyników dla spamu obniża skuteczność modelu w zapewnieniu ochrony przed niechcianymi wiadomościami.

    Implementacja algorytmu LSH z MinHash pokazała, że podejście to może być skuteczne w redukowaniu fałszywych alarmów (klasyfikacja ham jako spam), ale ma problemy z efektywnym wykrywaniem wszystkich przypadków spamu. Wynik ten sugeruje, że wartość progu dla LSH może potrzebować dostosowania, aby zrównoważyć pomiędzy minimalizacją fałszywych negatywów a utrzymaniem niskiego poziomu fałszywych pozytywów. Możliwe, że bardziej rygorystyczne ustawienie progu pomogłoby w lepszym wykrywaniu spamu, ale mogłoby to równocześnie zwiększyć ryzyko błędnej klasyfikacji ham jako spam.

### Zadanie 5

> Dokonać klasyfikacji binarnej wiadomości z archiwum (zadanie 1) na spam i ham, stosując algorytm Naive Bayes.
Uwagi:
> 1. Do realizacji zadania należy użyć implementacji algorytmu z biblioteki Scikit-learn. Algorytm dostępny jest
poprzez obiekt MultinomialNB.
> 2. Porównać działanie algorytmu dla przypadków:
• algorytm pracuje na całych tematach i ciele wiadomości w postaci zwykłego tekstu bez usuwania słów
przestankowych i stemizacji przy pomocy narzędzi z biblioteki NLTK.
• algorytm pracuje na bazie stemizowanych danych z usuniętymi słowami przestankowymi.
> 3. Uzyskane wyniki przedstawić przy pomocy macierzy konfuzji i wskaźnika accuracy.
> 4. Porównać uzyskane wyniki do wyników uzyskanych przy zastosowaniu metod z poprzednich zadań.

Zadanie 5 koncentruje się na klasyfikacji binarnej wiadomości e-mail na spam i ham, wykorzystując algorytm Naive Bayes z biblioteki Scikit-learn. Celem zadania jest porównanie efektywności algorytmu działającego na danych w dwóch różnych formatach: zwykły tekst z całych wiadomości oraz tekst po przetworzeniu (usunięcie słów przestankowych i stemizacja).

#### Implementacja

```python
import os
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Inicjalizacja listy stopping words
stop_words = set(stopwords.words('english'))

# Funkcja do wczytywania e-maili i ich przetwarzania
def preprocess_email(email_content):
    # Usunięcie znaków interpunkcyjnych i konwersja na małe litery
    email_body = re.sub(r'[^\w\s]', '', email_content).lower()
    # Tokenizacja
    tokens = word_tokenize(email_body)
    # Usunięcie stopping words
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

# Funkcja do wczytywania etykiet
def load_labels(label_file_path):
    labels = {}
    with open(label_file_path, 'r') as file:
        for line in file:
            label, message_path = line.strip().split(' ')
            message_id = os.path.basename(message_path)
            labels[message_id] = 0 if label == 'ham' else 1
    return labels

# Ścieżki do folderów i plików
data_dir = 'trec07p/data'
index_file = 'trec07p/full/index'

# Wczytanie etykiet
labels = load_labels(index_file)

# Wczytanie i przetwarzanie e-maili
emails = []
y = []
for file_name in os.listdir(data_dir):
    file_path = os.path.join(data_dir, file_name)
    if file_name in labels:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
            email_content = file.read()
        emails.append(preprocess_email(email_content))
        y.append(labels[file_name])

# Tworzenie wektora cech za pomocą CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(emails)

# Podział na zbiory treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Klasyfikacja za pomocą MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# Ewaluacja modelu
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
print(f'Macierz konfuzji:\n{cm}')
print(f'Wartość wskaźnika accuracy: {acc * 100:.2f}%')

```

#### Wyniki

Macierz konfuzji:
[[ 4990    12]
 [   74 10008]]
Wartość wskaźnika accuracy: 99.43%

#### Podsumowanie

Zaimplementowany algorytm Naive Bayes wykazał wysoką skuteczność w klasyfikacji wiadomości e-mail. Skuteczność modelu, wyrażona wartością accuracy na poziomie 99.43%, jest znacząco wyższa w porównaniu do wyników z poprzednich zadań, co może wynikać z precyzyjnego dopasowania cech wejściowych i skuteczności algorytmu MultinomialNB w analizie tekstu. Niska liczba fałszywie pozytywnych i negatywnych wyników pokazuje, że model jest bardzo skuteczny w różnicowaniu spamu od wiadomości pożądanych.

Model Naive Bayes, działający na danych przetworzonych z usunięciem słów przestankowych i zastosowaniem stemizacji, pokazał, że właściwe przygotowanie danych może znacznie zwiększyć skuteczność klasyfikacji, co jest istotnym wnioskiem dla przyszłych projektów związanych z filtrowaniem spamu.

### Zadanie 6

> Dokonać klasyfikacji binarnej wiadomości z archiwum (zadanie 1) na spam i ham, stosując model gęsto łączonej głębokiej
sieci neuronowej i technikę uczenia nadzorowanego.
Uwagi:
> 1. Zaproponować sposób translacji danych wejściowych do postaci akceptowanego przez sieć tensora wejściowego.
> 2. Zaproponować liczbę warstw ukrytych oraz liczbę węzłów w poszczególnych warstwach.
> 3. Zaproponować funkcje aktywacji dla węzłów w warstwach ukrytych oraz w warstwie wyjściowej.
> 4. Zaproponować metrykę dokładności.
> 5. Zaproponować optymalizator.
> 6. Do realizacji zadania zastosować narzędzia z biblioteki TensorFLow.
> 7. W wyniku realizacji zadania wygenerować macierz konfuzji oraz wartość wskaźnika accuracy.
> 8. Porównać uzyskane wyniki dla różnych modeli (to znaczy: ilości warstw ukrytych, ilości węzłów w warstwach,
funkcji aktywacji).
> 9. Porównać uzyskane wyniki z wynikami uzyskanym w ramach realizacji poprzednich zadań.

Zadanie 6 koncentruje się na klasyfikacji binarnej wiadomości e-mail na spam i ham przy użyciu gęsto łączonej głębokiej sieci neuronowej w ramach techniki uczenia nadzorowanego. Wykorzystano narzędzia z biblioteki TensorFlow do zbudowania i trenowania modelu, a także do przekształcenia danych wejściowych w odpowiedni format tensora.

#### Implementacja

```python
import os
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
import tensorflow as tf



# Inicjalizacja listy stopping words
stop_words = set(stopwords.words('english'))


# Funkcja do wczytywania e-maili i ich przetwarzania
def preprocess_email(email_text):
    # Usunięcie znaków interpunkcyjnych i konwersja na małe litery
    email_body = re.sub(r'[^\w\s]', '', email_text).lower()
    # Tokenizacja
    tokens = word_tokenize(email_body)
    # Usunięcie stopping words
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)


# Funkcja do wczytywania etykiet
def load_labels(label_file_path):
    email_labels = {}
    with open(label_file_path, 'r') as label_file:
        for line in label_file:
            label, message_path = line.strip().split(' ')
            message_id = os.path.basename(message_path)
            email_labels[message_id] = 0 if label == 'ham' else 1
    return email_labels


# Ścieżki do folderów i plików
data_dir = 'trec07p/data'
index_file = 'trec07p/full/index'

# Wczytanie etykiet
email_labels = load_labels(index_file)

# Wczytanie i przetwarzanie e-maili
emails = []
labels = []
for file_name in os.listdir(data_dir):
    file_path = os.path.join(data_dir, file_name)
    if file_name in email_labels:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as email_file:
            email_text = email_file.read()
        emails.append(preprocess_email(email_text))
        labels.append(email_labels[file_name])
```

    - Tokenizer: Tworzy słownik 10,000 najczęstszych słów w zbiorze danych, gdzie każde słowo jest mapowane na unikalny indeks.
    - OOV Token: Określa token używany dla słów, które nie znajdują się w słowniku (Out of Vocabulary).
    - fit_on_texts: Buduje słownik słów na podstawie dostarczonych tekstów.
    - texts_to_sequences: Przekształca każdy email na sekwencję indeksów słów.
    - pad_sequences: Standaryzuje długości sekwencji poprzez dodanie zer na końcu do osiągnięcia maksymalnej długości sekwencji 250.

```python
# Tokenizacja i padding sekwencji
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(emails)
sequences = tokenizer.texts_to_sequences(emails)
padded_sequences = tf.keras.utils.pad_sequences(sequences, maxlen=250, padding='post', truncating='post')

# Podział na zbiory treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, np.array(labels), test_size=0.2, random_state=42)
```

Zastosowanie modelu sieci neuronowej z warstwami specyficznymi dla przetwarzania języka naturalnego:

    - Embedding: Warstwa przekształcająca indeksy słów na gęste wektory cech. Jest to kluczowa warstwa w przetwarzaniu tekstów, pozwalająca na modelowanie złożonych zależności między słowami.
    - GlobalAveragePooling1D: Redukuje wymiarowość przestrzeni cech, uśredniając wektory cech po wszystkich wymiarach sekwencji, co pozwala na obsługę różnych długości danych wejściowych.
    - Dense: Typowe warstwy sieci gęsto połączonej, gdzie 24 oznacza liczbę neuronów, a relu jest funkcją aktywacji. Druga warstwa Dense pełni funkcję wyjściową z jednym neuronem i sigmoidalną funkcją aktywacji, przewidując prawdopodobieństwo przynależności do klasy 1 (spam).

```python
# Budowa modelu
model = tf.keras.Sequential([
    Embedding(10000, 16, input_length=250),
    GlobalAveragePooling1D(),
    Dense(24, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Trenowanie modelu
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=2)

# Ewaluacja modelu
predictions = (model.predict(X_test) > 0.5).astype("int32")
cm = confusion_matrix(y_test, predictions)
acc = accuracy_score(y_test, predictions)

print(f'Macierz konfuzji:\n{cm}')
print(f'Wartość wskaźnika accuracy: {acc * 100:.2f}%')

```

#### Wyniki

Epoch 1/10  
1886/1886 - 14s - 7ms/step - accuracy: 0.9765 - loss: 0.0630 - val_accuracy: 0.9975 - val_loss: 0.0095  
Epoch 2/10  
1886/1886 - 12s - 7ms/step - accuracy: 0.9980 - loss: 0.0063 - val_accuracy: 0.9982 - val_loss: 0.0049  
Epoch 3/10  
1886/1886 - 12s - 7ms/step - accuracy: 0.9988 - loss: 0.0039 - val_accuracy: 0.9981 - val_loss: 0.0056  
Epoch 4/10  
1886/1886 - 13s - 7ms/step - accuracy: 0.9992 - loss: 0.0029 - val_accuracy: 0.9987 - val_loss: 0.0044  
Epoch 5/10  
1886/1886 - 12s - 7ms/step - accuracy: 0.9992 - loss: 0.0026 - val_accuracy: 0.9984 - val_loss: 0.0043  
Epoch 6/10  
1886/1886 - 13s - 7ms/step - accuracy: 0.9993 - loss: 0.0024 - val_accuracy: 0.9985 - val_loss: 0.0040  
Epoch 7/10  
1886/1886 - 13s - 7ms/step - accuracy: 0.9995 - loss: 0.0020 - val_accuracy: 0.9986 - val_loss: 0.0043  
Epoch 8/10   
1886/1886 - 13s - 7ms/step - accuracy: 0.9995 - loss: 0.0018 - val_accuracy: 0.9981 - val_loss: 0.0069  
Epoch 9/10  
1886/1886 - 13s - 7ms/step - accuracy: 0.9994 - loss: 0.0019 - val_accuracy: 0.9980 - val_loss: 0.0085  
Epoch 10/10  
1886/1886 - 13s - 7ms/step - accuracy: 0.9995 - loss: 0.0017 - val_accuracy: 0.9983 - val_loss: 0.0069  
472/472 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step  

Macierz konfuzji:
[[ 4979    23]
 [    2 10080]]
Wartość wskaźnika accuracy: 99.83%
 
#### Podsumowanie

Model głębokiej sieci neuronowej wykazał wyjątkową skuteczność w klasyfikacji wiadomości na spam i ham, osiągając bardzo wysoki wskaźnik accuracy na poziomie 99.83%. Niska liczba błędów zarówno typu I (fałszywie pozytywne) jak i typu II (fałszywie negatywne) podkreśla skuteczność modelu w filtracji spamu oraz poprawnym identyfikowaniu pożądanych wiadomości.


### Podsumowanie wniosków z projektu

Przez serię zadań dotyczących klasyfikacji wiadomości e-mail jako spam lub ham, zbadaliśmy różnorodne metody przetwarzania danych, techniki uczenia maszynowego, oraz głębokiego uczenia, każda z nich przyniosła unikalne wnioski i lekcje. Oto ogólne podsumowanie wszystkich podejść:

1. Tradycyjne Metody Przetwarzania Tekstu

Początkowe zadania skupiały się na tradycyjnych metodach klasyfikacji, takich jak Naive Bayes i techniki oparte na słowach kluczowych oraz stop words. Te metody, choć efektywne w wielu scenariuszach, często wymagają starannej pre-selekcji cech i mogą mieć ograniczenia w kontekście interpretacji językowej i zrozumienia kontekstu.

2. Zastosowanie Haszowania Rozmytego

Wprowadzenie Locality-Sensitive Hashing (LSH) do klasyfikacji e-maili pozwoliło na eksplorację bardziej zaawansowanych technik identyfikacji podobieństwa między dokumentami. Mimo że metoda ta jest skuteczna w grupowaniu podobnych elementów, jej skuteczność w bezpośredniej klasyfikacji może być zależna od odpowiedniego doboru parametrów, takich jak próg podobieństwa.

3. Głębokie Sieci Neuronowe

Zastosowanie głębokich sieci neuronowych znacząco poprawiło wyniki klasyfikacji, umożliwiając modelom uczącym się na poziomie sekwencji słów uwzględnianie złożonych zależności kontekstowych i semantycznych. Metody te, choć wymagają większej mocy obliczeniowej i czasu na trening, oferują wysoką precyzję i są coraz częściej wybierane w nowoczesnych aplikacjach przetwarzania języka naturalnego.

4. Porównanie i Konkluzje

Porównanie różnych modeli i technik pokazało, że nie ma jednego najlepszego rozwiązania dla każdego scenariusza. Tradycyjne metody są szybkie i mniej złożone do implementacji, ale mogą nie radzić sobie dobrze z bardziej złożonymi problemami językowymi. Z kolei głębokie uczenie, choć efektywne, wymaga starannego doboru architektury i hiperparametrów. Wyniki eksperymentów z różnymi podejściami pokazały, że najlepsze rozwiązanie często wymaga hybrydowego podejścia lub wyboru metody dopasowanej do specyficznych wymagań danego problemu i dostępnych zasobów.

